## Drive Mount

In [1]:
base_path = "/content/drive/My Drive/yl_tez"

In [2]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers==4.41.2
!pip install datasets
!pip install trl

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from trl import SFTTrainer, SFTConfig

In [ ]:
def find_last_checkpoint(output_dir):
    if not os.path.exists(output_dir) or not os.path.isdir(output_dir):
        return None
    checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir, d))]
    checkpoints = [d for d in checkpoints if 'checkpoint' in d]
    if len(checkpoints) == 0:
        return None
    return max(checkpoints, key=os.path.getmtime)

# Değişkenler

In [ ]:
model_name = "ytu-ce-cosmos/turkish-gpt2-medium"
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)
veri_seti_adi =  "random_templated_sohbetler20000.csv"
data_filepath = os.path.join(base_path, veri_seti_adi)
new_special_tokens = ["<|USER|>", "<|ASSISTANT|>", "<|DOCTOR_TITLE|>", "<|SPECIALITY|>", "</s>"]

# Veri setini oku

In [ ]:
# Veri dosyasını oku
df = pd.read_csv(data_filepath)

In [ ]:
print(df.head().to_markdown())
len(df)

In [ ]:
dataset = Dataset.from_pandas(df)
train_test_split = dataset.train_test_split(test_size=0.1, seed=42) # seed parametresi rastgelelik olmasın diye)

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [ ]:
print(dataset)

# Tokenizer Güncelleme

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({"additional_special_tokens": new_special_tokens})
tokenizer.pad_token = tokenizer.eos_token

# Model Eğitme

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
training_args = SFTConfig(
    output_dir=model_yolu,  # Modelin kaydedileceği dizin
    report_to="none",  # Raporlama aracı
    dataloader_drop_last=True,  # Son batch'i bırak
    per_device_eval_batch_size=8,  # Değerlendirme batch boyutu
    per_device_train_batch_size=8, # Her cihaz başına eğitim batch boyutu
    save_total_limit=3,  # En fazla 3 kontrol noktası sakla,,
    save_steps=2000,  # Kaydetme adımı, save_strategy="epoch" için gerek yok
    eval_steps=2000,  # Test etme adımı eval_strategy="epoch" için gerek yok
    logging_steps=2000,  # Loglama adımı logging_strategy="epoch" için gerek yok steps için var
    eval_strategy="steps",  # Değerlendirme stratejisi adım bazında
    save_strategy="steps",  # Kaydetme stratejisi adım bazında
    logging_strategy="steps",
    num_train_epochs=20,  # Eğitim epoch sayısı,
    gradient_checkpointing=False,  # Gradient checkpointing
    gradient_accumulation_steps=8,  # Gradient accumulation steps,
    fp16=False,  # 16-bit float training
    max_seq_length=512,  # Maksimum girdi uzunluğu
)

In [ ]:
trainer = SFTTrainer(
        model,
        tokenizer=tokenizer,
        args=training_args,
        train_dataset=dataset,
        eval_dataset=eval_dataset,
        dataset_text_field="text", # veri setinin hangi elemanını kullanacak
        max_seq_length=512, # context length
    )


In [ ]:
if find_last_checkpoint(model_yolu) is not None:
    trainer.train(resume_from_checkpoint=True)
else:
    trainer.train()

Modeli Kaydet

In [ ]:
trainer.save_model()

Logları kaydet

In [ ]:
import json
# Eğitim ve değerlendirme metriklerini kaydetme
log_history = trainer.state.log_history
log_file = os.path.join(model_yolu, "training_log.json")
with open(log_file, "w") as f:
    json.dump(log_history, f)